# Homework 9

In this homework, we'll deploy the dogs vs cats model we trained in the previous homework.

Download the model from here:

https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats-model/dogs_cats_10_0.687.h5

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the converted model?

In [1]:
import os
import tensorflow as tf

model = tf.keras.models.load_model('dogs_cats_10_0.687.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
fout_name = 'dogs_cats_10_0.687.tflite' 

with open(fout_name, 'wb') as fout:
    fout.write(tflite_model)
    
print(f'File size: {os.path.getsize(fout_name) / 1024**2:.0f}MB')

INFO:tensorflow:Assets written to: g:\cache\tmpdir\tmpv2epapt1\assets


File size: 43MB


## Question 2

To be able to use this model, we need to know the index of the input and the index of the output.

What's the output index for this model?

In [2]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.get_output_details()[0]['index']

13

## Preparing the image

You'll need some code for downloading and resizing images. You can use this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have pillow installed:

```
pip install pillow
```

Let's download and resize this image:

https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg

Based on the solution of the previous homework, what should be the target size for the image?

In [3]:
input_shape = tuple(interpreter.get_input_details()[0]['shape'][1:])
input_shape

(150, 150, 3)

## Question 3

Now we need to turn the image into an numpy array and pre-process it.

> Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

In [4]:
from io import BytesIO
from urllib import request

from PIL import Image

import numpy as np

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

url = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'
img = prepare_image(download_image(url), input_shape[:2])
img_array = np.asarray(img).astype(np.float32) / 255

img_array[0][0][0].round(7)

0.7058824

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [5]:
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, [img_array])
interpreter.invoke()

prediction = interpreter.get_tensor(output_index)
prediction[0][0].round(7)

0.7699092

## Prepepare the lambda code

Now you need to copy all the code into a separate python file. You will need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook by importing the file and invoking the function from this file.
Docker

For the next two questions, we'll use a Docker image that I already prepared. This is the Dockerfile that I used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.8
COPY cats-dogs-v2.tflite .
```

And pushed it to agrigorev/zoomcamp-cats-dogs-lambda:v2.

> Note: The image already contains a model and it's not the same model as the one we used for questions 1-4.

## Question 5

Now let's extend this docker image, install all the required libraries and add the code for lambda.

You don't need to include the model in the image. It's already included. The name of the file with the model is cats-dogs-v2.tflite and it's in the current workdir in the image (see the Dockerfile above for the reference).

What's the image id of the base image?

In the build logs (on Linux), you'll see a log like that:

```
$ docker some-command-for-building
Sending build context to Docker daemon  2.048kB
Step 1/N : FROM agrigorev/zoomcamp-cats-dogs-lambda:v2
 ---> XXXXXXXXXXXX
Step 2/N : ....
```

You need to get this `XXXXXXXXXXXX`.

On MacOS and Windows, the logs for `docker build` are different. To get the image id there, you can use `docker image ls -a`.

In [6]:
!docker image ls -a

REPOSITORY                            TAG       IMAGE ID       CREATED          SIZE
zoomcamp-cats-dogs-lambda             v2        030e6069d9fa   18 minutes ago   702MB
agrigorev/zoomcamp-cats-dogs-lambda   v2        322fc756f258   3 days ago       608MB


## Question 6

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/commons/1/18/Vombatus_ursinus_-Maria_Island_National_Park.jpg

What's the output from the model?

In [7]:
!docker run zoomcamp-cats-dogs-lambda:v2

[0.54134727]
